In [ ]:
# conferencia

In [22]:
# number of clusters
    # definido como 3 pq conheço o iris dataset
K = 3

# fuzziness value 
m = [1.1, 1.6, 2]
# os valores definidos acima foram indicados pelo professor

# cálculo com base na função do artigo que o professor mostrou
def calcular_sigma(iris_data_without_last_column):
    n = iris_data_without_last_column.shape[0]
    total_distance = 0

    # bora por partes pq nem eu e nem o gpt somos deus
    # soma pra todos os pares de distância
    for i in range(n):
        for j in range(n):
            total_distance += np.linalg.norm(iris_data_without_last_column[i] - iris_data_without_last_column[j])**2

    # média e raiz
    sigma = np.sqrt(total_distance / (2 * n**2))
    return sigma

# calcular
sigma = calcular_sigma(iris_data_without_last_column)
print("Valor de sigma:", sigma)

U = np.random.rand(iris_data_without_last_column.shape[0], K)

def calculando_centroides(iris_data_without_last_column, K, U, m):
    centroides = np.zeros((K, iris_data_without_last_column.shape[1]))
    for i in range(K):
        U_corrigido = np.tile(U[:, i][:, np.newaxis], (1, iris_data_without_last_column.shape[1]))
        m_expandido = np.tile(m[i], (iris_data_without_last_column.shape[0], iris_data_without_last_column.shape[1]))
        centroides[i, :] = np.sum((U_corrigido ** m_expandido) * iris_data_without_last_column, axis=0) / np.sum(U_corrigido ** m_expandido)
    return centroides

centroides = calculando_centroides(iris_data_without_last_column, K, U, m)

def gaussian_kernel(iris_data_without_last_column, centroides, sigma):
    """
    EQUAÇÃO DO KERNEL TRICK
    
    """
    num_points = iris_data_without_last_column.shape[0]
    num_centroides = centroides.shape[0]
    kernel_m = np.zeros((num_points, num_centroides))

    for i in range(num_points):
        for j in range(num_centroides):
            distance_squared = np.sum((iris_data_without_last_column[i] - centroides[j]) ** 2)
            kernel_m[i, j] = np.exp(-0.5 * distance_squared / sigma**2) # -0.5 como o -1/2 antes que eu surte e ache que tá errado de novo

    return kernel_m

kernel_m = gaussian_kernel(iris_data_without_last_column, centroides, sigma)

def kg_membership(iris_data_without_last_column, centroides, sigma):
    """
    Calcula a matriz de membership fuzzy entre os dados e os centroides usando o kernel gaussiano.

    Args:
    iris_data_without_last_column (ndarray): Conjunto de dados.
    centroides (ndarray): Matriz de centroides dos clusters.
    sigma (float): parâmetro de largura!!!!! muito importante futuramente (sim preciso escrever isso em todo lugar para quando a ansiedade bater)
   
   Returns:
    ndarray: matriz de MEMBERSHIP fuzzy
        # lembrando que o pertencimento com kernel é calculado com base nas distâncias
        # então aqui meio que substitui a matriz_dg do FCM
    """
    num_points = iris_data_without_last_column.shape[0]
    num_centroides = centroides.shape[0]
    U_novo = np.zeros((num_points, num_centroides))

    for i in range(num_points):
        for j in range(num_centroides):
            x = iris_data_without_last_column[i:i+1]  # Seleciona um ponto de dados como um vetor unidimensional
            c = centroides[j:j+1]  # Seleciona um centróide como um vetor unidimensional
            U_novo[i, j] = gaussian_kernel(x, c, sigma)

    return U_novo # novos valores de pertinência agora foram calculados com KERNEL

U_novo =  kg_membership(iris_data_without_last_column, centroides, sigma)

def matriz_dg(iris_data_without_last_column, centroides):
    num_points = iris_data_without_last_column.shape[0]
    num_centroides = centroides.shape[0]

    distancias = np.zeros((num_points, num_centroides))

    for i in range(num_points):
        for j in range(num_centroides):
            distancias[i, j] = np.linalg.norm(iris_data_without_last_column[i] - centroides[j])

    return distancias

matriz_dg = matriz_dg(iris_data_without_last_column, centroides)

# mudando a função objetivo pra considerar o kernel
def fobj_kernel(U_novo, kernel_matrix, m):
    # os valores da função vão sendo somados 
    
    c, n = U_novo.shape
    objetivo = 0

    for i in range(c):
        for j in range(n):
            objetivo += U_novo[i, j] ** m * kernel_matrix[i, j]

    return objetivo

N = iris_data_without_last_column.shape[0] # número de dados
velho_obj = fobj_kernel(U_novo, matriz_dg, m)


aximo = 100

# Inicializar valores de pertinência e centroides
U = np.random.rand(iris_data_without_last_column.shape[0], K)
centroides = calculando_centroides(iris_data_without_last_column, K, U, m)

# Calcular matriz de distâncias inicial
J_anterior = matriz_dg


print("Centroides iniciais:")
print(centroides)
print()
print("Membership iniciais:")
print(U)
print()
print('J:')
print(J_anterior)
print()


for iteration in range(maximo):
    # Atualizar valores de pertinência e centroides
    U_novo = kg_membership(iris_data_without_last_column, centroides, sigma)
    centroides = calculando_centroides(iris_data_without_last_column, K, U_novo, m)
    
    # Calcular nova matriz de distâncias
    nova_J = matriz_dg
    
    # Calcular função objetivo
    novo_objetivo = fobj_kernel(U_novo, nova_J, m)

    # Verificar convergência
    if (np.all(nova_J <= J_anterior) or 
        np.all(np.abs(novo_objetivo - velho_obj) < 1e-6) or
        iteration == maximo - 1):
        break

    # Atualizar valores para a próxima iteração
    velho_obj = novo_objetivo
    J_anterior = nova_J

    # Atualizar pertinências para a próxima iteração
    U = U_novo
    
    
print("Centroides finais:")
print(centroides)
print()
print("Membership final:")
print(U)
print()
print('J final:')
print(nova_J)
print()